In [ ]:
# Transfer learning and model-generated feature embeddings ... under construction

In [17]:
import os
import numpy as np
import librosa
import tensorflow as tf

In [2]:
from math import floor
from lib_augment import load_and_normalize
from lib_feature import feature_whistleness

%matplotlib inline
import matplotlib.pyplot as plt


In [3]:
# deploy_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz'
species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}
fs = 48000
time_reso = 0.02
hop_length = int(time_reso*fs)  #
species_list = list(species_dict.keys())
species_id = list(species_dict.values())

In [4]:
species_to_code = {'bottlenose': 'BD', 'longbeaked_common': 'CD', 'shortbeaked_common': 'CD', 'common': 'CD',
                   'striped': 'STR', 'spotted': 'SPT', 'spinner': 'SPIN', 'pilot': 'PLT', 'roughtoothed': 'RT',
                   'false_killer': 'FKW'}

In [5]:
# species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}

step_sec = 1.0  # sec
whistle_ratio = 0.2  # percentage of whislteness frames in a long window
# step_size = int(1.0*fs)
win_long_sec = 10   # 10 sec
step_ind = int(win_long_sec/step_sec)
step_size = int(step_sec*fs)
win_size = int(2.0*fs)

In [6]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'

In [10]:
# loading the trained model
from tensorflow.keras.models import load_model
import tensorflow_addons.layers.spatial_pyramid_pooling as spp


model_whistleness_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_whistleness/2021-02-02_200514_resnet18_expt_alldata_run0_f1_lr_0.001/epoch_163_valloss_0.2434_valacc_0.9472.hdf5'
# model_whistleness = load_model(model_whistleness_path)
# model_whistleness.summary()

# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210303_233728/STAR2000/epoch_53_valloss_0.9505_valacc_0.7552.hdf5'
# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210305_105231/STAR2000/epoch_56_valloss_0.1945_valacc_0.7305.hdf5'
model_species_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210310_095616/STAR2000/epoch_48_valloss_0.1938_valacc_0.7403.hdf5'

model_species = load_model(model_species_path)
model_species.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 100, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 128, 16)      160       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 128, 16)      64        
_________________________________________________________________
activation (Activation)      (None, 100, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 128, 16)      2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 128, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 100, 128, 16)      0     

In [8]:
sound_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald'
sound_deploy = ['STAR2000_48kHz', 'STAR2003_48kHz', 'STAR2006_48kHz', 'HICEAS2002_48kHz', 'PICEAS2005_48kHz']

## STAR 2000

In [9]:
# np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)
feature_curr = np.load(os.path.join(ten_sec_path, 'STAR2000_48kHz_test_others_train.npz'))
feature_train = feature_curr['feature_train']
feature_test = feature_curr['feature_test']
label_train = feature_curr['label_train']
label_test = feature_curr['label_test']
del feature_curr

In [14]:
model_species.layers[53]

In [15]:
model_species.layers[-9]

In [19]:
model_species.trainable = False

In [18]:
model_long = tf.keras.Sequential()